# 🧪 Create Assay Nodes & Relationships

This notebook reads the dataset manifest, extracts assay and material metadata, maps tissues/cells to UBERON/Cell Ontology, and writes Neo4j CSVs for assay nodes and their relationships to studies, anatomy, cell types, genes, and methylation regions using `genelab_utils` and `ontology_mapper`.

Author: Peter W. Rose, UC San Diego (pwrose.ucsd@gmail.com)

In [34]:
import os
import pandas as pd
import ontology_mapper
import genelab_utils as gl

In [35]:
pd.set_option('display.max_rows', None)  # Shows all rows
pd.set_option('display.max_colwidth', None)  # Shows full content of each cell

## Setup Environment Variables
Edit `../.env` to configure the environment.  

In [36]:
# Node and relationship directory paths
node_dir, rel_dir = gl.setup_environment()

Environment setup for KG version: v0.0.3


In [37]:
# Edit `../.env` to define `BIOPORTAL_API_KEY` for ontology lookups. 
apikey = os.getenv("BIOPORTAL_API_KEY")
if not apikey:
    raise Exception("BIOPORTAL_API_KEY is not set in the .env file!")

In [38]:
MANIFEST_PATH = "../data/manifest.csv"

## Get Info about available Datasets

In [39]:
manifest = pd.read_csv(MANIFEST_PATH, dtype=str, keep_default_na=False,
                       usecols=["identifier", "technology", "measurement", "assay_name", "organism", "material", "filename"])
# Remove duplicates that have multiple materials. These will be extracted from the factors to ensure a proper mapping
manifest.drop_duplicates(subset=["identifier", "technology", "measurement", "assay_name", "filename"], inplace=True)
manifest.head()

identifier                         technology                measurement  \
0    OSD-100           RNA Sequencing (RNA-Seq)    transcription profiling   
1    OSD-101           RNA Sequencing (RNA-Seq)    transcription profiling   
2    OSD-102           RNA Sequencing (RNA-Seq)    transcription profiling   
3    OSD-103  Whole Genome Bisulfite Sequencing  DNA methylation profiling   
4    OSD-103           RNA Sequencing (RNA-Seq)    transcription profiling   

                                                                     assay_name  \
0                      OSD-100_transcription-profiling_rna-sequencing-(rna-seq)   
1             OSD-101_transcription-profiling_rna-sequencing-(rna-seq)_Illumina   
2  OSD-102_transcription-profiling_rna-sequencing-(rna-seq)_Illumina HiSeq 4000   
3           OSD-103_dna-methylation-profiling_whole-genome-bisulfite-sequencing   
4                      OSD-103_transcription-profiling_rna-sequencing-(rna-seq)   

       organism            material  \
0  Mus musculus            left eye   
1  Mus musculus  Left gastrocnemius   
2  Mus musculus         Left kidney   
3  Mus musculus     Quadriceps-left   
4  Mus musculus     Quadriceps-left   

                                                        filename  
0                   GLDS-100_rna_seq_differential_expression.csv  
1                   GLDS-101_rna_seq_differential_expression.csv  
2                   GLDS-102_rna_seq_differential_expression.csv  
3  GLDS-103_Gwgbs_differential_methylation_tiles_GLMethylSeq.csv  
4                   GLDS-103_rna_seq_differential_expression.csv

## Extract Assay Data

In [40]:
variables = {"transcription profiling": "Log2fc_",
             "DNA methylation profiling": "meth.diff_",
            }

In [41]:
assays = gl.extract_assay_info(manifest, variables)
assays.head()

identifier                technology              measurement  \
0    OSD-100  RNA Sequencing (RNA-Seq)  transcription profiling   
0    OSD-100  RNA Sequencing (RNA-Seq)  transcription profiling   
1    OSD-101  RNA Sequencing (RNA-Seq)  transcription profiling   
1    OSD-101  RNA Sequencing (RNA-Seq)  transcription profiling   
2    OSD-102  RNA Sequencing (RNA-Seq)  transcription profiling   

                                                                     assay_name  \
0                      OSD-100_transcription-profiling_rna-sequencing-(rna-seq)   
0                      OSD-100_transcription-profiling_rna-sequencing-(rna-seq)   
1             OSD-101_transcription-profiling_rna-sequencing-(rna-seq)_Illumina   
1             OSD-101_transcription-profiling_rna-sequencing-(rna-seq)_Illumina   
2  OSD-102_transcription-profiling_rna-sequencing-(rna-seq)_Illumina HiSeq 4000   

       organism            material  \
0  Mus musculus            left eye   
0  Mus musculus            left eye   
1  Mus musculus  Left gastrocnemius   
1  Mus musculus  Left gastrocnemius   
2  Mus musculus         Left kidney   

                                       filename  \
0  GLDS-100_rna_seq_differential_expression.csv   
0  GLDS-100_rna_seq_differential_expression.csv   
1  GLDS-101_rna_seq_differential_expression.csv   
1  GLDS-101_rna_seq_differential_expression.csv   
2  GLDS-102_rna_seq_differential_expression.csv   

                           factors         factors_1         factors_2  
0  (Ground Control)v(Space Flight)  [Ground Control]    [Space Flight]  
0  (Space Flight)v(Ground Control)    [Space Flight]  [Ground Control]  
1  (Ground Control)v(Space Flight)  [Ground Control]    [Space Flight]  
1  (Space Flight)v(Ground Control)    [Space Flight]  [Ground Control]  
2  (Ground Control)v(Space Flight)  [Ground Control]    [Space Flight]

## Extract Material Information and Map to UBERON Ontology

In [42]:
materials = gl.extract_materials(assays)
# Remove rows where 'factors' contains any digit followed by space (e.g., 1 day) since they cannot match any cell or tissue types
materials = materials[~materials["material"].str.contains(r'\d ', na=False)]
print("Number of materials to map:", materials.shape[0])

Number of materials to map: 258


In [43]:
mapped_materials = ontology_mapper.map_ontology(materials, "material", "material", "UBERON", apikey)
mapped_materials = mapped_materials[mapped_materials["material_id"] != ""].reset_index()

In [44]:
mapped_materials[["material", "material_name", "material_id", "material_uri"]]

material              material_name  \
0                                   3D Cells                       cell   
1                                   3D cells                       cell   
2                              Adrenal gland              adrenal gland   
3                 Adrenal glands- both sides              adrenal gland   
4                                      Blood                      blood   
5                                Bone Marrow                bone marrow   
6                                      Brain                      brain   
7                                      Cells                       cell   
8                                   Cerebrum                   cerebrum   
9                                      Colon                      colon   
10                             Cortical Bone              cortical bone   
11     Extensor digitorum longus- both sides  extensor digitorum longus   
12                             Gastrocnemius              gastrocnemius   
13                       Gastrocnemius-right              gastrocnemius   
14                                     Heart                      heart   
15  Human Peripheral blood lymphocytes (PBL)           blood lymphocyte   
16                                    Kidney                     kidney   
17                    Left Lobe of the Liver    liver left lateral lobe   
18                                 Left Lung                  left lung   
19                        Left gastrocnemius              gastrocnemius   
20                               Left kidney                left kidney   
21                   Left quadriceps femoris         quadriceps femoris   
22                               Left retina                     retina   
23                                     Liver                      liver   
24                                      Lung                       lung   
25                           Mandibular bone                   mandible   
26                                 Myoblasts                   myoblast   
27                     Peripheral blood cell                 blood cell   
28                           Primary T Cells                     t cell   
29                        Quadriceps femoris         quadriceps femoris   
30                           Quadriceps-left                 quadriceps   
31                                    Retina                     retina   
32           Right extensor digitorum longus  extensor digitorum longus   
33                              Right kidney               right kidney   
34                              Right retina                     retina   
35                              Right soleus                     soleus   
36                   Right tibialis anterior          tibialis anterior   
37                                    Soleus                     soleus   
38                         Soleus-both sides                     soleus   
39                                    Spleen                     spleen   
40                             Spleen-distal                     spleen   
41                             Temporal Bone              temporal bone   
42                                    Thymus                     thymus   
43                    Tibialis anterior-left          tibialis anterior   
44                          Vastus Lateralis           vastus lateralis   
45                               Whole Blood                whole blood   
46                              Whole Spleen                     spleen   
47                                    Zygote                     zygote   
48                             adrenal gland              adrenal gland   
49                                     brain                      brain   
50                     cardiac muscle tissue             cardiac muscle   
51                                     cells                       cell   
52                          descending colon           descen

## Create Anatomy (proxy) Nodes

In [45]:
material_ids = mapped_materials[["material_id"]].copy()
material_ids.rename(columns={"material_id": "identifier"}, inplace=True)
anatomy = material_ids[material_ids["identifier"].str.startswith("UBERON:")].copy()

In [46]:
anatomy_nodes = gl.save_dataframe_to_kg(anatomy, 'Anatomy', node_dir)
print(f"Number of Anatomy nodes: {anatomy_nodes.shape[0]}")
anatomy_nodes.head()

Number of Anatomy nodes: 33


identifier
2  UBERON:0002369
4  UBERON:0000178
5  UBERON:0002371
6  UBERON:0000955
8  UBERON:6110636

## Create CellType (proxy) Nodes

In [47]:
cell_type = material_ids[material_ids["identifier"].str.startswith("CL:")].copy()

In [48]:
cell_type_nodes = gl.save_dataframe_to_kg(cell_type, 'CellType', node_dir)
print(f"Number of CellType nodes: {cell_type_nodes.shape[0]}")
cell_type_nodes.head()

Number of CellType nodes: 7


identifier
0   CL:0000000
15  CL:4030029
26  CL:0000056
27  CL:0000081
28  CL:0000084

## Assign Materials for Assay Factors 1 and 2

In [49]:
assays = gl.assign_material_to_assays(assays, mapped_materials)

## Assign Unique Assay Identifier

In [50]:
assays = gl.add_assay_identifiers(assays)

## Create Study-PERFORMED_SpA-Assay Relationships

In [51]:
study_performed_assay = assays[["study_id", "identifier"]].copy()
study_performed_assay.rename(columns={"study_id": "from", "identifier": "to"}, inplace=True)

In [52]:
study_performed_assay_rels = gl.save_dataframe_to_kg(study_performed_assay, 'Study-PERFORMED_SpAS-Assay', rel_dir)
print(f"Number of Study-PERFORMED_SpAS-Assay relationships: {study_performed_assay_rels.shape[0]}")
study_performed_assay_rels.head()

Number of Study-PERFORMED_SpAS-Assay relationships: 6185


from                                        to
0  OSD-100  OSD-100-daab5c089f2a23b4d18cbf45d113535b
0  OSD-100  OSD-100-0ee3281ad117571bed3fd0b4543f80ea
1  OSD-101  OSD-101-59e73a75c7ba08b2907095b0ca9e57e8
1  OSD-101  OSD-101-fefc0a3bd4b859b1e035ced363f4e08f
2  OSD-102  OSD-102-f2039e8d02fe8eb80f4edc65677b0f77

## Create Assay-INVESTIGATED_AiA-Anatomy Relationships

In [53]:
assay_investigated_material_1 = assays[["identifier", "material_id_1"]].copy()
assay_investigated_material_1.rename(columns={"identifier": "from", "material_id_1": "to"}, inplace=True)

assay_investigated_material_2 = assays[["identifier", "material_id_2"]].copy()
assay_investigated_material_2.rename(columns={"identifier": "from", "material_id_2": "to"}, inplace=True)

assay_investigated_material = pd.concat([assay_investigated_material_1, assay_investigated_material_2]).drop_duplicates()

assay_investigated_anatomy = assay_investigated_material[assay_investigated_material["to"].str.startswith("UBERON:")]

In [54]:
assay_investigated_anatomy_rels = gl.save_dataframe_to_kg(assay_investigated_anatomy, 'Assay-INVESTIGATED_ASiA-Anatomy', rel_dir)
print(f"Number of Assay-INVESTIGATED_ASiA-Anatomy relationships: {assay_investigated_anatomy_rels.shape[0]}")
assay_investigated_anatomy_rels.head()

Number of Assay-INVESTIGATED_ASiA-Anatomy relationships: 6071


from              to
0  OSD-100-daab5c089f2a23b4d18cbf45d113535b  UBERON:0004548
0  OSD-100-0ee3281ad117571bed3fd0b4543f80ea  UBERON:0004548
1  OSD-101-59e73a75c7ba08b2907095b0ca9e57e8  UBERON:0001388
1  OSD-101-fefc0a3bd4b859b1e035ced363f4e08f  UBERON:0001388
2  OSD-102-f2039e8d02fe8eb80f4edc65677b0f77  UBERON:0004538

## Create Assay-INVESTIGATED_AiCT-CellType Relationships

In [55]:
assay_investigated_cell_type = assay_investigated_material[assay_investigated_material["to"].str.startswith("CL:")]

In [56]:
assay_investigated_cell_type_rels = gl.save_dataframe_to_kg(assay_investigated_cell_type, 'Assay-INVESTIGATED_ASiCT-CellType', rel_dir)
print(f"Number of Assay-INVESTIGATED_ASiCT-CellType: {assay_investigated_cell_type_rels.shape[0]}")
assay_investigated_cell_type_rels.head()

Number of Assay-INVESTIGATED_ASiCT-CellType: 1134


from          to
8  OSD-109-311bb16f4e456a48c4f1bc555878f9fd  CL:0000000
8  OSD-109-24d7925b9a73c1ecb5e18e44174fd043  CL:0000000
8  OSD-109-b647183b7061e4076f2e0251f406ad48  CL:0000000
8  OSD-109-850e09fdbf94fc58e5ef83b1c6cf5d64  CL:0000000
8  OSD-109-810f01e6c3e983ca673d6ce6606e6e9e  CL:0000000

## Create Assay Nodes

In [57]:
assays.rename(columns={'assay_name': 'name'}, inplace=True)
assay_props = assays[["identifier", "name", "technology", "measurement", 
                      "factors_1", "factors_2", 
                      "material_1", "material_2", 
                      "material_name_1", "material_name_2",
                      "material_id_1", "material_id_2"
                     ]].copy()

assay_nodes = gl.save_dataframe_to_kg(assay_props, 'Assay', node_dir)
print(f"Number of Assays nodes: {assay_nodes.shape[0]}")
assay_nodes.head()

Number of Assays nodes: 6185


identifier  \
0  OSD-100-daab5c089f2a23b4d18cbf45d113535b   
0  OSD-100-0ee3281ad117571bed3fd0b4543f80ea   
1  OSD-101-59e73a75c7ba08b2907095b0ca9e57e8   
1  OSD-101-fefc0a3bd4b859b1e035ced363f4e08f   
2  OSD-102-f2039e8d02fe8eb80f4edc65677b0f77   

                                                                           name  \
0                      OSD-100_transcription-profiling_rna-sequencing-(rna-seq)   
0                      OSD-100_transcription-profiling_rna-sequencing-(rna-seq)   
1             OSD-101_transcription-profiling_rna-sequencing-(rna-seq)_Illumina   
1             OSD-101_transcription-profiling_rna-sequencing-(rna-seq)_Illumina   
2  OSD-102_transcription-profiling_rna-sequencing-(rna-seq)_Illumina HiSeq 4000   

                 technology              measurement       factors_1  \
0  RNA Sequencing (RNA-Seq)  transcription profiling  Ground Control   
0  RNA Sequencing (RNA-Seq)  transcription profiling    Space Flight   
1  RNA Sequencing (RNA-Seq)  transcription profiling  Ground Control   
1  RNA Sequencing (RNA-Seq)  transcription profiling    Space Flight   
2  RNA Sequencing (RNA-Seq)  transcription profiling  Ground Control   

        factors_2          material_1          material_2 material_name_1  \
0    Space Flight            left eye            left eye        left eye   
0  Ground Control            left eye            left eye        left eye   
1    Space Flight  Left gastrocnemius  Left gastrocnemius   gastrocnemius   
1  Ground Control  Left gastrocnemius  Left gastrocnemius   gastrocnemius   
2    Space Flight         Left kidney         Left kidney     left kidney   

  material_name_2   material_id_1   material_id_2  
0        left eye  UBERON:0004548  UBERON:0004548  
0        left eye  UBERON:0004548  UBERON:0004548  
1   gastrocnemius  UBERON:0001388  UBERON:0001388  
1   gastrocnemius  UBERON:0001388  UBERON:0001388  
2     left kidney  UBERON:0004538  UBERON:0004538

In [58]:
assay_measured_mgene = gl.extract_transcription_data(assays, threshold=0.05)

processing: OSD-100
processing: OSD-101
processing: OSD-102
processing: OSD-103
processing: OSD-104
processing: OSD-105
processing: OSD-109
No statistically significant data for OSD-109: Log2fc_(Fe-56 ion radiation & 1 day)v(Fe-56 ion radiation & 3 day)
No statistically significant data for OSD-109: Log2fc_(Fe-56 ion radiation & 1 day)v(sham-irradiated & 1 day)
No statistically significant data for OSD-109: Log2fc_(Fe-56 ion radiation & 3 day)v(Fe-56 ion radiation & 1 day)
No statistically significant data for OSD-109: Log2fc_(sham-irradiated & 1 day)v(Fe-56 ion radiation & 1 day)
processing: OSD-117
No statistically significant data for OSD-117: Log2fc_(proton & 1 day)v(proton & 12 day)
No statistically significant data for OSD-117: Log2fc_(proton & 1 day)v(proton & 26 day)
No statistically significant data for OSD-117: Log2fc_(proton & 1 day)v(proton & 5 day)
No statistically significant data for OSD-117: Log2fc_(proton & 1 day)v(sham-irradiated & 1 day)
No statistically significant 

## Create Assay-MEASURED_AmMG-MGene Nodes

In [59]:
assay_measured_mgene_rels = gl.save_dataframe_to_kg(assay_measured_mgene, 'Assay-MEASURED_ASmMG-MGene', rel_dir)
print(f"Number of Assay-MEASURED_ASmMG-MGene relationships: {assay_measured_mgene_rels.shape[0]}")
assay_measured_mgene_rels.head()

Number of Assay-MEASURED_ASmMG-MGene relationships: 25878862


from      to    log2fc  adj_p_value
0  OSD-100-daab5c089f2a23b4d18cbf45d113535b   23849  0.211489     0.001988
1  OSD-100-daab5c089f2a23b4d18cbf45d113535b  235339  0.355894     0.007801
2  OSD-100-daab5c089f2a23b4d18cbf45d113535b   12444  0.300122     0.040228
3  OSD-100-daab5c089f2a23b4d18cbf45d113535b   66108  0.293361     0.023020
4  OSD-100-daab5c089f2a23b4d18cbf45d113535b   57278  0.320432     0.024716

In [60]:
methylation_data = gl.extract_methylation_data(assays, threshold=0.05)

processing: OSD-103
processing: OSD-105
processing: OSD-47
processing: OSD-48


## Create MethylationRegion Nodes

In [61]:
methylation_data["name"] = methylation_data["methylation_id"]
methylation_region = methylation_data[["methylation_id", "name", "chr", "start", "end", "dist.to.feature", "in_promoter", "in_exon", "in_intron"]].copy()
methylation_region.rename(columns={"methylation_id": "identifier", "chr": "chromosome", "dist.to.feature": "dist_to_feature"}, inplace=True)
methylation_region["dist_to_feature"] = methylation_region["dist_to_feature"].astype(int)

In [62]:
methylation_region_nodes = gl.save_dataframe_to_kg(methylation_region, 'MethylationRegion', node_dir)
print(f"Number of MethylationRegion nodes: {methylation_region_nodes.shape[0]}")
methylation_region_nodes.head()

Number of MethylationRegion nodes: 5663


identifier                 name chromosome     start       end  \
0  1:17167001-17168000  1:17167001-17168000          1  17167001  17168000   
1  1:21022001-21023000  1:21022001-21023000          1  21022001  21023000   
2  1:24496001-24497000  1:24496001-24497000          1  24496001  24497000   
3  1:34341001-34342000  1:34341001-34342000          1  34341001  34342000   
4  1:36308001-36309000  1:36308001-36309000          1  36308001  36309000   

   dist_to_feature in_promoter in_exon in_intron  
0              114        true    true      true  
1               91        true    true      true  
2            -4176       false   false      true  
3             7476       false    true      true  
4             3517       false    true      true

## Create Assay-MEASURED_AmMR-MethylationRegion Relationships

In [63]:
assay_measured_methylation_region = methylation_data[["assay_id", "methylation_id", "methylation_diff", "q_value"]].copy()
assay_measured_methylation_region.rename(columns={"assay_id": "from", "methylation_id": "to"}, inplace=True)

In [64]:
assay_measured_methylation_region_rel = gl.save_dataframe_to_kg(assay_measured_methylation_region, 'Assay-MEASURED_ASmMR-MethylationRegion', rel_dir)
print(f"Number of Assay-MEASURED_ASmMR-MethylationRegion relationships: {assay_measured_methylation_region_rel.shape[0]}")
assay_measured_methylation_region_rel.head()

Number of Assay-MEASURED_ASmMR-MethylationRegion relationships: 9555


from                   to  \
0  OSD-103-07920f1ed30d671670e714edffc6f250  1:17167001-17168000   
1  OSD-103-07920f1ed30d671670e714edffc6f250  1:21022001-21023000   
2  OSD-103-07920f1ed30d671670e714edffc6f250  1:24496001-24497000   
3  OSD-103-07920f1ed30d671670e714edffc6f250  1:34341001-34342000   
4  OSD-103-07920f1ed30d671670e714edffc6f250  1:36308001-36309000   

   methylation_diff       q_value  
0          8.671846  2.030562e-03  
1         11.667655  2.470766e-02  
2         -9.373073  3.356339e-02  
3        -10.682111  2.283094e-02  
4        -28.820344  2.347983e-14

## Create MGene-METHYLATED_IN_MGmMR-MethylationRegion Relationships

In [65]:
mgene_methylated_in_methylation_region = methylation_data[["ENTREZID", "methylation_id"]].copy()
mgene_methylated_in_methylation_region.rename(columns={"ENTREZID": "from", "methylation_id": "to"}, inplace=True)

In [66]:
mgene_methylated_in_methylation_region_rels = gl.save_dataframe_to_kg(mgene_methylated_in_methylation_region, 'MGene-METHYLATED_IN_MGmMR-MethylationRegion', rel_dir)
print(f"Number of MGene-METHYLATED_IN_MGmMR-MethylationRegion relationships: {mgene_methylated_in_methylation_region_rels.shape[0]}")
mgene_methylated_in_methylation_region_rels.head()

Number of MGene-METHYLATED_IN_MGmMR-MethylationRegion relationships: 5694


from                   to
0   57339  1:17167001-17168000
1   71877  1:21022001-21023000
2   12823  1:24496001-24497000
3   13518  1:34341001-34342000
4  214854  1:36308001-36309000